### กลุ่ม Diabetes Prediction 
1. 6610402205 นายรักษิต รุ่งรัตนไชย หมู่ 1
2. 6610402132 นายบวรรัตน์ ตั้งนรารัชชกิจ หมู่ 1
3. 6610401985 นายไชยวัตน์ หนูวัฒนา หมู่ 1 

### วัตถุประสงค์ของระบบต้นแบบ
กลุ่มของพวกเราเล็งเห็นถึงความสำคัญทางด้านสุขภาพของประชาชนคนไทย ในปัจจุบันคนไทยประสบกับปัญหาสุขภาพมากมายแต่หนึ่งปัญหาที่สร้างความเสียหายให้กับสุขภาพ และคนไทยเป็นกันมากที่สุดนั่นคือโรคเบาหวาน พฤติกรรมการใช้ชีวิตของคนไทยในสมัยนี้ก็เป็นปัจจัยหนึ่งที่ทำให้ปริมาณผู้ป่วยโรคเบาหวานเพิ่มมากยิ่งขึ้น พวกเราจึงอยากทำระบบต้นแบบที่ช่วยชี้แนะหรือแนะนำผู้ที่มาใช้งานระบบต้นแบบของเราว่ามีความเสี่ยงที่จะเป็นโรคเบาหวานหรือไม่เพื่อให้เกิดการตระหนักและระมัดระวังในพฤติกรรมการใช้ชีวิตของตนเองซึ่งระบบต้นแบบของเราไม่ได้แม่นยำ 100 เปอร์เซ็นต์ ไม่สามารถใช้วินิจฉัยโรคได้แต่ใช้ประเมินความเสี่ยงเบื่องต้นของผู้ที่มาใช้งานระบบได้หากมีความเสี่ยงก็สามารถให้แพทย์วินิจัยอย่างถูกต้องและแม่นยำต่อไป

### ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ
Link to data: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

In [1]:
import pandas as pd

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")
diabetes.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


### ปั​ญห​า ML ของตั​วเอ​งเป็นปั​ญหา​ป​ระเภ​ทใด (classificati​on/regres​sion/clust​ering/ge​neration)
ปัญหา ML ของกลุ่มพวกเราเป็นปัญหาแบบ classification binary เนื่องจากเป็นการทำนายว่าข้อมูลของบุคคลที่นำเข้ามาในโมเดลนั้นเป็นหรือไม่เป็นโรคเบาหวานจึงเป็นการ classification binary อย่างชัดเจน

### f​eat​ur​es ที่ใช้ในการทำนา​ยมีอะ​ไรบ้าง
ก่อนที่เราจะสามารถดูค่าความสำคัญของแต่ละฟีเจอร์ได้นั้น เราต้องทำ Feature Engineer กับข้อมูลให้เรียบร้อยก่อนโดยมีขั้นตอนดังต่อไปนี้

แยกชุดข้อมูลออกเป็น ชุดข้อมูลเรียนรู้(train) ชุดข้อมูลทดสอบ(test) และชุดข้อมูลตรวจสอบความแม่นยำของโมเดล(validation)

In [2]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, stratify=diabetes["Diabetes_binary"]
                               , test_size=0.3, random_state=1234)
test, validation = train_test_split(test, stratify=test["Diabetes_binary"],
                                    test_size=0.3, random_state=1234)
len(train), len(test), len(validation)

(177576, 53272, 22832)

เนื่องจากข้อมูลของกลุ่มเรานั้น ขาดความสมดุลอย่างมากสังเกตได้จาก จำนวนของ Label ที่มีจำนวนไม่เท่ากัน

In [3]:
diabetes.Diabetes_binary.value_counts()

Diabetes_binary
0.0    218334
1.0     35346
Name: count, dtype: int64

ดังนั้นต้องทำให้ชุดข้อมูลที่นำมาใช้งานนั้นมีความสมดุลกันก่อนด้วยการทำ Under Sampling

In [4]:
from imblearn.under_sampling import RandomUnderSampler
resampler = RandomUnderSampler(random_state=1234)

feature = diabetes.drop(columns="Diabetes_binary").columns
label = "Diabetes_binary"

train[feature], train[label] = resampler.fit_resample(train[feature], train[label])
train.dropna(inplace=True)

train.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
186497,1.0,1.0,0.0,1.0,29.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,5.0,2.0,6.0,1.0,1.0,13.0,3.0,2.0
18001,0.0,0.0,0.0,1.0,25.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,6.0,8.0
37591,0.0,0.0,0.0,1.0,25.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,8.0,6.0,8.0
119797,0.0,1.0,1.0,1.0,30.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,9.0,4.0,7.0
55137,1.0,1.0,0.0,1.0,37.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,15.0,20.0,1.0,0.0,5.0,5.0,5.0


ขั้นตอนต่อมาคือการทำ Pipeline ของ Logistic Regression, SDG Classifier และ Decision Tree โดยเหตุผลที่เลือกทำ pipeline ทั้ง 3 โมเดลเพราะ ต้องการหาโมเดลที่มีความแม่นยำสูงและไม่ Overfit จนเกินไป

สร้าง Pipeline ของ Logistic Regression

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression

def create_pipline_logistic():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        LogisticRegression(max_iter=500)
    )

logistic_pipeline = create_pipline_logistic()
logistic_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer-1',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI']),
                                                 ('kbinsdiscretizer-2',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['PhysHlth']),
                                                 ('kbinsdiscretizer-3',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['MentHlth'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

สร้างโมเดล Logistic Regression และตรวจสอบความแม่นยำ

In [6]:
logistic_pipeline.fit(train[feature], train[label])
logistic_pipeline.score(validation[feature], validation[label])

0.7362473721093202

สร้าง Pipeline ของ SDG Classifier

In [7]:
from sklearn.linear_model import SGDClassifier

def create_pipline_sgd():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        SGDClassifier()
    )

sgd_pipline = create_pipline_sgd()
sgd_pipline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer-1',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI']),
                                                 ('kbinsdiscretizer-2',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['PhysHlth']),
                                                 ('kbinsdiscretizer-3',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['MentHlth'])])),
                ('sgdclassifier', SGDClassifier())])

สร้างโมเดล SDG Classifier และตรวจสอบความแม่นยำ

In [8]:
sgd_pipline.fit(train[feature], train[label])
sgd_pipline.score(validation[feature], validation[label])

0.5847494744218641

สร้าง Pipeline ของ Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier

def create_pipline_decisiontree():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        DecisionTreeClassifier()
    )

decision_pipeline = create_pipline_decisiontree()
decision_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer-1',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI']),
                                                 ('kbinsdiscretizer-2',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['PhysHlth']),
                                                 ('kbinsdiscretizer-3',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['MentHlth'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

สร้างโมเดล Decision Tree และตรวจสอบความแม่นยำ

In [10]:
decision_pipeline.fit(train[feature], train[label])
decision_pipeline.score(validation[feature], validation[label])

0.6752365101611773

ตรวจสอบค่าความสำคัญของแต่ละฟีเจอร์จากโมเดล Logistic Regression เพราะเป็นโมเดลมีความแม่นยำสูงที่สุด

In [11]:
from sklearn.inspection import permutation_importance

importance = permutation_importance(logistic_pipeline, test[feature], test[label], random_state=1234)
importance = pd.Series(importance.importances_mean, index=feature)
importance.sort_values(ascending=False)

GenHlth                 0.008000
BMI                     0.002729
HeartDiseaseorAttack    0.000894
DiffWalk                0.000548
HvyAlcoholConsump       0.000218
Stroke                  0.000120
NoDocbcCost             0.000034
Smoker                 -0.000026
AnyHealthcare          -0.000053
PhysActivity           -0.000056
Veggies                -0.000263
MentHlth               -0.000282
Income                 -0.000428
PhysHlth               -0.000548
Education              -0.000638
Fruits                 -0.000856
CholCheck              -0.001010
Sex                    -0.001490
HighChol               -0.003792
Age                    -0.005001
HighBP                 -0.008905
dtype: float64

โดยเลือก 7 ฟีเจอร์แรกที่มีค่าความสำคัญสูงสุดมาใช้งาน

In [12]:
importance_features = ["GenHlth", "BMI", "HeartDiseaseorAttack", "DiffWalk", "HvyAlcoholConsump", "Stroke", "NoDocbcCost"]

### เฉ​ลย (label) ที่ต้​องกา​รทำนา​ยคื​ออ​ะไร
สิ่งที่ต้องการทำนายคือ 0 หรือ 1 (0 คือไม่เป็นโรคเบาหวานและ 1 คือเป็นโรคเบาหวาน)

In [13]:
labels = ["ไม่เป็นโรคเบาหวาน", "เป็นโรคเบาหวาน"]

for i in diabetes.Diabetes_binary.unique().astype(int):
    print(f"{i} = {labels[i]}")

0 = ไม่เป็นโรคเบาหวาน
1 = เป็นโรคเบาหวาน


### ระบบของตัวเองสาม​ารถใช้ al​gorith​m อะไรได้บ้าง เพ​ราะเห​ตุใด

เกณฑ์ที่กลุ่มเราใช้ในการเลือกใช้โมเดลจะอิงจาก Cheat Sheet ของ Scikit-Learn ดังภาพต่อไปนี้:

<img src="https://scikit-learn.org/1.3/_static/ml_map.png" alt="sklearn-cheat-sheet" width="700">

เนื่องจากชุดข้อมูลที่กลุ่มเรานำมาใช้นั้นเป็นแบบ Classification หรือแบ่งแยกประเภท และมีเป็นข้อมูลที่มี Labeled จากรูปจะมี `SGD Classifier` และ `Kernel Approximation` ที่สามารถนำมาใช้กับข้อมูลของกลุ่มเราได้ และยังมี Algorithm อื่นๆอีกที่สามารถนำมาใช้งานกับข้อมูลของกลุ่มเราได้แต่ไม่ได้อยู่ใน Cheat Sheet นี้เช่น `Logistic Regression` และ `Decision Tree` เป็นต้น

### แต่ละ algor​ithm มี hype​rpar​am​eter อะไ​รบ้าง ปรับเป็​นอะไรไ​ด้บ้าง
แต่ละ algorithm มี hyperparameter ดังต่อไปนี้

In [14]:
logistic_pipeline.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('kbinsdiscretizer-1',
                                    KBinsDiscretizer(encode='ordinal'), ['BMI']),
                                   ('kbinsdiscretizer-2',
                                    KBinsDiscretizer(encode='ordinal',
                                                     strategy='uniform'),
                                    ['PhysHlth']),
                                   ('kbinsdiscretizer-3',
                                    KBinsDiscretizer(encode='ordinal',
                                                     strategy='uniform'),
                                    ['MentHlth'])])),
  ('logisticregression', LogisticRegression(max_iter=500))],
 'verbose': False,
 'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('kbinsdiscretizer-1',
                                  KBinsDiscretizer(e

In [15]:
sgd_pipline.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('kbinsdiscretizer-1',
                                    KBinsDiscretizer(encode='ordinal'), ['BMI']),
                                   ('kbinsdiscretizer-2',
                                    KBinsDiscretizer(encode='ordinal',
                                                     strategy='uniform'),
                                    ['PhysHlth']),
                                   ('kbinsdiscretizer-3',
                                    KBinsDiscretizer(encode='ordinal',
                                                     strategy='uniform'),
                                    ['MentHlth'])])),
  ('sgdclassifier', SGDClassifier())],
 'verbose': False,
 'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('kbinsdiscretizer-1',
                                  KBinsDiscretizer(encode='ordinal'), ['BM

In [16]:
decision_pipeline.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('kbinsdiscretizer-1',
                                    KBinsDiscretizer(encode='ordinal'), ['BMI']),
                                   ('kbinsdiscretizer-2',
                                    KBinsDiscretizer(encode='ordinal',
                                                     strategy='uniform'),
                                    ['PhysHlth']),
                                   ('kbinsdiscretizer-3',
                                    KBinsDiscretizer(encode='ordinal',
                                                     strategy='uniform'),
                                    ['MentHlth'])])),
  ('decisiontreeclassifier', DecisionTreeClassifier())],
 'verbose': False,
 'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('kbinsdiscretizer-1',
                                  KBinsDiscretizer(encod

ทำการ Tunning ให้กับโมเดล Logistic Regresion เพื่อหา hyperparameter ที่สามารถปรับเพื่อทำให้โมเดลมีประสิทธิภาพมากขึ้น

In [17]:
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")
pipeline = create_pipline_logistic() 

param_dist = {
     'columntransformer__kbinsdiscretizer-1__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer-2__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer-3__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer-1__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer-2__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer-3__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer-1__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-2__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-3__n_bins': [5, 10, 20],
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logisticregression__max_iter': [100, 200, 300],
    'logisticregression__tol': [1e-4, 1e-3, 1e-2]
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=20, random_state=0)
search.fit(train[feature], train[label])
search.best_params_, f"{search.best_score_:.3f}"

({'logisticregression__tol': 0.0001,
  'logisticregression__solver': 'liblinear',
  'logisticregression__penalty': 'l1',
  'logisticregression__max_iter': 200,
  'logisticregression__C': 100,
  'columntransformer__kbinsdiscretizer-3__strategy': 'uniform',
  'columntransformer__kbinsdiscretizer-3__n_bins': 20,
  'columntransformer__kbinsdiscretizer-3__encode': 'ordinal',
  'columntransformer__kbinsdiscretizer-2__strategy': 'kmeans',
  'columntransformer__kbinsdiscretizer-2__n_bins': 10,
  'columntransformer__kbinsdiscretizer-2__encode': 'ordinal',
  'columntransformer__kbinsdiscretizer-1__strategy': 'kmeans',
  'columntransformer__kbinsdiscretizer-1__n_bins': 10,
  'columntransformer__kbinsdiscretizer-1__encode': 'onehot-dense'},
 '0.746')

In [18]:
search.best_estimator_.score(validation[feature], validation[label])

0.7376489138051857

ปรับแต่ง hyperparameter ตามผลที่ได้จากการ tunning model

ทำการ Tunning ให้กับโมเดล SGD Classifier เพื่อหา hyperparameter ที่สามารถปรับเพื่อทำให้โมเดลมีประสิทธิภาพมากขึ้น

In [19]:
warnings.filterwarnings("ignore")
pipeline = create_pipline_sgd()

param_dist = {
    'columntransformer__kbinsdiscretizer-1__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-2__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-3__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-1__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-2__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-3__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-1__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-2__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-3__n_bins': [5, 10, 20],
    'sgdclassifier__loss' : ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'sgdclassifier__penalty': ['l2', 'l1', 'elasticnet', None]
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=20, random_state=0)
search.fit(train[feature], train[label])
search.best_params_, f"{search.best_score_:.3f}"

({'sgdclassifier__penalty': 'l2',
  'sgdclassifier__loss': 'log_loss',
  'columntransformer__kbinsdiscretizer-3__strategy': 'uniform',
  'columntransformer__kbinsdiscretizer-3__n_bins': 5,
  'columntransformer__kbinsdiscretizer-3__encode': 'onehot',
  'columntransformer__kbinsdiscretizer-2__strategy': 'uniform',
  'columntransformer__kbinsdiscretizer-2__n_bins': 5,
  'columntransformer__kbinsdiscretizer-2__encode': 'onehot-dense',
  'columntransformer__kbinsdiscretizer-1__strategy': 'uniform',
  'columntransformer__kbinsdiscretizer-1__n_bins': 10,
  'columntransformer__kbinsdiscretizer-1__encode': 'onehot'},
 '0.733')

In [20]:
search.best_estimator_.score(validation[feature], validation[label])

0.7237648913805186

ทำการ Tunning ให้กับโมเดล Decision Tree เพื่อหา hyperparameter ที่สามารถปรับเพื่อทำให้โมเดลมีประสิทธิภาพมากขึ้น

In [21]:
warnings.filterwarnings("ignore")
pipeline = create_pipline_decisiontree()

param_dist = {
    'columntransformer__kbinsdiscretizer-1__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-2__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-3__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-1__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-2__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-3__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-1__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-2__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-3__n_bins': [5, 10, 20],
    'decisiontreeclassifier__criterion':['gini', 'entropy', 'log_loss'],
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=20, random_state=34)
search.fit(train[feature], train[label])
search.best_params_, f"{search.best_score_:.3f}"

({'decisiontreeclassifier__criterion': 'entropy',
  'columntransformer__kbinsdiscretizer-3__strategy': 'kmeans',
  'columntransformer__kbinsdiscretizer-3__n_bins': 20,
  'columntransformer__kbinsdiscretizer-3__encode': 'onehot-dense',
  'columntransformer__kbinsdiscretizer-2__strategy': 'kmeans',
  'columntransformer__kbinsdiscretizer-2__n_bins': 5,
  'columntransformer__kbinsdiscretizer-2__encode': 'onehot',
  'columntransformer__kbinsdiscretizer-1__strategy': 'quantile',
  'columntransformer__kbinsdiscretizer-1__n_bins': 5,
  'columntransformer__kbinsdiscretizer-1__encode': 'ordinal'},
 '0.660')

In [22]:
search.best_estimator_.score(validation[feature], validation[label])

0.6748423265592152

### คำตอบที่ได้จากการทำนายของ model เบื้องต้นปกติหรือไม่ อย่างไร ต้องมีการ p​os​t-p​roc​ess หรือไม่ อย่างไร
เริ่มด้วยการ
1. ทดสอบประสิทธิภาพของโมเดลเบื้องต้น 
2. นำโมเดลเบื้องต้นมาทำนายเพื่อดูความแม่นยำ

In [23]:
test_efficient_logistic = create_pipline_logistic()
%time test_efficient_logistic.fit(train[feature], train[label])
test_efficient_logistic.score(validation[feature], validation[label])

CPU times: total: 125 ms
Wall time: 219 ms


0.7362473721093202

In [24]:
samples = validation.sample(5, random_state=1234)
samples

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
251504,0.0,1.0,0.0,1.0,30.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,6.0,5.0
237617,0.0,0.0,0.0,1.0,33.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0
167201,0.0,1.0,1.0,1.0,31.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,3.0,0.0,1.0,0.0,1.0,5.0,4.0,4.0
57989,0.0,0.0,0.0,1.0,21.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,6.0,7.0
32578,0.0,1.0,1.0,1.0,26.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,1.0,0.0,13.0,4.0,6.0


In [25]:
predicted = logistic_pipeline.predict(samples)
predicted

array([0., 0., 1., 0., 1.])

ผลจากการทำนายเบื้องต้นพบว่ายังคงมีการทำนายผิดพลาดเกิดขึ้นอยู่จึงต้องทำการ post process เพื่อเพิ่มความแม่นยำของโมเดลโดยการ drop บางฟีเจอร์ที่มีความสำคัญน้อยออกไปใช้เฉพาะฟีเจอร์ที่มีความสำคัญมาก (ดูความสำคัญของฟีเจอร์จากการทำ feature importance ก่อนหน้านี้)

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression

def create_new_pipline_logistic():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            remainder="passthrough"
        ),
        LogisticRegression(max_iter=500)
    )

new_logistic_pipeline = create_new_pipline_logistic()
new_logistic_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [27]:
new_logistic_pipeline.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('kbinsdiscretizer',
                                    KBinsDiscretizer(encode='ordinal'), ['BMI'])])),
  ('logisticregression', LogisticRegression(max_iter=500))],
 'verbose': False,
 'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('kbinsdiscretizer',
                                  KBinsDiscretizer(encode='ordinal'), ['BMI'])]),
 'logisticregression': LogisticRegression(max_iter=500),
 'columntransformer__force_int_remainder_cols': True,
 'columntransformer__n_jobs': None,
 'columntransformer__remainder': 'passthrough',
 'columntransformer__sparse_threshold': 0.3,
 'columntransformer__transformer_weights': None,
 'columntransformer__transformers': [('kbinsdiscretizer',
   KBinsDiscretizer(encode='ordinal'),
   ['BMI'])],
 'columntransformer__verbose': False,
 'columntransformer__verbose_feature_names_o

In [28]:
%time new_logistic_pipeline.fit(train[importance_features], train[label])
new_logistic_pipeline.score(validation[importance_features], validation[label])

CPU times: total: 15.6 ms
Wall time: 50.3 ms


0.7143920812894183

In [29]:
warnings.filterwarnings("ignore")
pipeline = create_new_pipline_logistic() 

param_dist = {
     'columntransformer__kbinsdiscretizer__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer__n_bins': [5, 10, 20],  # Adjusted to correct key
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logisticregression__max_iter': [100, 200, 300],
    'logisticregression__tol': [1e-4, 1e-3, 1e-2],
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=30, random_state=0)
search.fit(train[importance_features], train[label])
search.best_params_, f"{search.best_score_:.3f}"

({'logisticregression__tol': 0.01,
  'logisticregression__solver': 'lbfgs',
  'logisticregression__penalty': 'l2',
  'logisticregression__max_iter': 300,
  'logisticregression__C': 0.1,
  'columntransformer__kbinsdiscretizer__strategy': 'quantile',
  'columntransformer__kbinsdiscretizer__n_bins': 10,
  'columntransformer__kbinsdiscretizer__encode': 'onehot-dense'},
 '0.710')

In [30]:
search.best_estimator_.score(validation[importance_features], validation[label])

0.7283199018920813

หลังจากการทำ post process โดยการทำ Hyperparameters Tuning พบว่า score ความแม่นยำลดลงล็กน้อยและ มีระยะเวลาที่ใช้ในการสร้างโมเดลที่เร็วขึ้น

### ปร​ะเด็น​อื่น ๆ ที่เกี่ยว​ข้องกั​บข้อมูลของ​ตัวเอง

* ในขั้นตอนการหาค่าความสำคัญของฟีเจอร์จากการทดสอบหาค่าความสำคัญจากโมเดล SDG Classifier พบว่าผลลัพธ์ที่ได้มีค่าเป็น 0.0 ทั้งหมด ไม่ทราบว่าเพราะเหตุผลใดจึงได้ผลลัพธ์ออกมาเป็นแบบนั้นจึงเลือกที่จะใช้แค่ 2 โมเดลนั่นคือ Logistic regression และ Decision tree ซึ่งทั้งสองโมเดลนี้สามารถหา feature importance ได้

* อีกประเด็นที่ค้นพบก็คือในตอนแรกข้อมูลเกิดความ imbalance ซึ่งคิดว่าไม่ได้ส่งผลอะไรเพราะตอนแบ่ง train test ได้ทำการแบ่งอย่างถูกต้องต้องแล้วตามที่งานก่อนหน้านี้ที่ได้ทำไปซึ่งผลการทดสอบโมเดลก็อยู่ในคะแนนที่ดีประมาณ 0.8 กว่าๆ แต่เมื่อทำการทดลองใช้โมเดลกับข้อมูลที่มีแต่คนที่เป็นโรคเบาหวานปรากฏว่าโมเดลทำนายไม่แม่นเลย score ลดลงเหลือ 0.1 กว่าๆแปลว่าโมเดลพยายามเดา label ให้เป็น 0 เพราะในชุด train มีจำนวน label 0 มากกว่า 1 หลายเท่าตัว จึงทำการแก้ imbalace โดยใช้วิธี `under sampling` ให้จำนวนคนเป็นกับไม่เป็นโรคเบาหวานเท่าๆกัน (ซึ่งแลกมาด้วยการที่ข้อมูลหายไปจำนวนมากจาก 2 แสนกว่า เหลือ 5 หมื่นกว่า) แต่สิ่งที่ได้มาคือโมเดลทำนายได้ดีขึ้นวัด score อยู่ที่ 0.7 นิดๆทั้งชุดที่มีแต่คนเป็นโรคและชุดของคนที่ไม่เป็นโรค (อัลกอริธึมที่ใช้คือ Logistic Regression)

### ก​าร​มีส่วน​ร่วมของสม​าชิกแต่ละ​คนในกลุ่ม (แต่ล​ะคนทำอะไรบ้าง)
- นายรักษิต รุ่งรัตนไชย ทำหน้าที่เขียนในหัวข้อ 1 2 3 4 8 9 10
- นายบวรรัตน์ ตั้งนรารัชชกิจ ทำหน้าที่เขียนในหัวข้อ 5 6 7 ทำการ tunning model เพื่อหา hyperparameter ที่ดีที่สุด
- นายไชยวัตน์ หนูวัฒนา เขียน pipeline เพื่อใช้ในการสร้าง model, หา feature importance และ tunning hyperparameter

### กา​รเปิ​ดเผ​ยกา​รใช้เค​รื่อ​งมือปัญ​ญาป​ระดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, pro​mpt อย่างไร)
ในการทำงานครั้งนี้ ไม่ได้ใช้ปัญญาประดิษฐ์ในการทำงานแต่อย่างใด